#### GAME PLAN
If it fails to scrape add those links to a reseed list to try again
    -> get more explicit in exception catching/case handling
Only go one click deep 
Set up Flask based website dating app
Hit Links to SQL
New Text to Mongo
Be better at scraping new text!

---------------
Need to update mongodb 'text' entries to use consistent webscraping mechanism, then check the follow url for javascript: 'http://www.datasciencecentral.com/profiles/blogs/how-you-can-improve-customer-experience-with-fast-data-analytics'  
  
This url uses xml and will be a good test point to see if JS is still coming through.  

---------

#### Need to handle pymongo timeout errors!

scrape as you go

In [1]:
import requests
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import re
import time
import pickle
from collections import defaultdict
import smtplib
import pandas as pd
from copy import deepcopy
from urllib.parse import urlparse
from pymongo import MongoClient
import pymongo
import os

In [41]:
password = os.environ['password']
libera_url = "54.164.158.211"

In [44]:
y = col.aggregate( [ {'$match': {'quality':True}},{'$sample' : {'size': 2}}])

In [45]:
x = y.next()

In [38]:
z = y.next()

In [46]:
x

{'_id': ObjectId('593f8ed9f7e49c258ed381bd'),
 'hand_reviewed': False,
 'inbound_link_count': 0,
 'open_seed': True,
 'previous_quality': True,
 'quality': True,
 'text': "// <![CDATA[\n        var disqus_shortname = 'bigdatamadesimple';\n        (function () {\n            var nodes = document.getElementsByTagName('span');\n            for (var i = 0, url; i < nodes.length; i++) {\n                if (nodes[i].className.indexOf('dsq-postid') != -1) {\n                    nodes[i].parentNode.setAttribute('data-disqus-identifier', nodes[i].getAttribute('rel'));\n                    url = nodes[i].parentNode.href.split('#', 1);\n                    if (url.length == 1) { url = url[0]; }\n                    else { url = url[1]; }\n                    nodes[i].parentNode.href = url + '#disqus_thread';\n                }\n            }\n            var s = document.createElement('script'); s.async = true;\n            s.type = 'text/javascript';\n            s.src = '//' + 'disqus.com/foru

In [43]:
for x in y:
    print(x)

In [25]:
y['url']

'http://101.datascience.community/tag/education/'

In [8]:
for x in y:
    print(x)

{'_id': ObjectId('59486713f7e49c0a9f7ba471'), 'url': 'http://abbottanalytics.blogspot.com/search/label/art', 'hand_reviewed': False, 'quality': False, 'open_seed': True, 'previous_quality': True}


In [ ]:
# class WebCrawler:
#     def __init__(self):
        
# #         self.mongo_password = os.environ['password']
# #         self.client = client = pymongo.MongoClient("mongodb://paul:" + password + "@" + ip_address + "/libera_db")
# #         self.db = self.client.libera_db
# #         self.col = self.db.scraped_
#     def get_soup(self, url):
#         """Takes a url and returns the BeautifulSoup object of the webpage"""
#         source_code = requests.get(url)
#         plain_text = source_code.text.encode('utf-8')
#         soup = BeautifulSoup(plain_text, "html.parser")
#         return soup
#         time.sleep(1)

In [ ]:
# Don't forget to start up mongodb on AWS instance!

### E-mail notice functions

In [3]:
def requests_error_notice(error_code,current_url):
    """Sends an e-mail notification if requests fails to connect to a url"""
    smtpObj = smtplib.SMTP('smtp.gmail.com', 587)
    smtpObj.ehlo()
    smtpObj.starttls()
    smtpObj.login('pfblack.utility@gmail.com', 'Metis1033')
    smtpObj.sendmail('pfblack.utility@gmail.com', 'paul.laifu.black@gmail.com',
                     'Subject: Get request failed\nRequests failed to scrape the following url\n \
                     url: %s \n \
                     error code: %i (if 0 requests failed to connect)' % (current_url, error_code))
    smtpObj.quit()

In [4]:
def duplicate_notice(db_type, db, col, url, duplicate_count):
    """Sends an e-mail notification if a url has been found in the database multiple times"""
    smtpObj = smtplib.SMTP('smtp.gmail.com', 587)
    smtpObj.ehlo()
    smtpObj.starttls()
    smtpObj.login('pfblack.utility@gmail.com', 'Metis1033')
    smtpObj.sendmail('pfblack.utility@gmail.com', 'paul.laifu.black@gmail.com',
                     'Subject: Duplicate Found\nA duplicate entry has been found on your %s database.\n \
                     Database: %s \n \
                     Collection: %s \n \
                     Url: %s \n \
                     Count: %i' % (db_type, db, col, url, duplicate_count))
    smtpObj.quit()

In [5]:
def termination_notice(location):
    """Sends an e-mail notification if function/process terminates/concludes
    
    Note: Requires that no unhandled errors were thrown and that the function concluded normally
    """
    smtpObj = smtplib.SMTP('smtp.gmail.com', 587)
    smtpObj.ehlo()
    smtpObj.starttls()
    smtpObj.login('pfblack.utility@gmail.com', 'Metis1033')
    smtpObj.sendmail('pfblack.utility@gmail.com', 'paul.laifu.black@gmail.com',
                     'Subject: Progam Stopped Running\nYour WebCrawler has terminated on %s!' % location)
    smtpObj.quit()

In [6]:
# Global Variables
client = pymongo.MongoClient("mongodb://paul:" + password + "@" + libera_url + "/libera_db")

db = client.libera_db

col = db.scraped_blogs

only_body_tag = SoupStrainer("body")

http_pattern = re.compile('https://www.|https://|http://www.|http://|www.')

location = 'local computer'

In [7]:
df = pickle.load(open("mongodb_data.p",'rb'))

In [26]:
urls = df['url']

In [33]:
urls[0]

'http://www.bschool.com/blog/2011/40-fascinating-blogs-for-the-ultimate-statistics-geek/'

In [38]:
len(quality)

3813

In [44]:
value = False

In [45]:
col.update_one({'url':'http://www.bschool.com/blog/2011/40-fascinating-blogs-for-the-ultimate-statistics-geek/'},{'$set':{'quality':value}})

In [49]:
for x in range(len(urls)):
    url = urls[x]
    value = bool(quality[x])
    col.update_one({'url':url},{'$set':{'quality':value}})

In [50]:
len(urls)

3813

In [29]:
quality = df['quality']

In [34]:
col.find({})[0]

{'_id': ObjectId('593f026ff7e49c1cd22aea40'),
 'hand_reviewed': False,
 'inbound_link_count': 1,
 'open_seed': False,
 'previous_quality': True,
 'quality': True,
 'text': '',
 'url': 'http://olomon.com/yuebofa.php',
 'urls_pointed_here': ['http://www.olomon.com/404.html']}

In [7]:
max_iterations = 1
def run_crawler(max_iterations, location):
    for _ in range(max_iterations):
        current_url = get_seed()
        source_code = make_request(current_url)
        if source_code:
            soup = BeautifulSoup(source_code.text, "lxml", parse_only=only_body_tag)
            get_links(soup, current_url)
        else:
            pass
    termination_notice(location)

In [8]:
def get_seed():
    """Grabs an open seed from the libera_db collection: scraped blogs
    and then sets the value of the open_seed field to False.
    
    $natural, 1 sets the order of scan to create order
    """
    current_url = col.find({'open_seed':True}).hint([('$natural', 1)]).limit(3)[2]['url']
    col.update_one({'url':current_url},{'$set':{'open_seed':False}})
    return current_url

In [9]:
# Used to pull up soup's plain text from seed AND to iterate through link_list
def make_request(current_url):
    """Takes a url and returns the BeautifulSoup object of the webpage"""
    try:
        source_code = requests.get(current_url)
        time.sleep(1)
        # If requests connects, but does not succeed in getting source_code, return None, e-mail error_notice, edit data entry
        if source_code.status_code != 200:
            error_code = source_code.status_code
            col.update_one({'url':current_url},{'$set':{'failed_scrape':True, 'error_code':source_code.status_code}})
            requests_error_notice(error_code, current_url)
            source_code = None
    # if request fails to connect, return None, e-mail error_notice, edit database entry
    except:
        error_code = 0
        requests_error_notice(error_code, current_url)
        col.update_one({'url':current_url},{'$set':{'failed_scrape':True, 'error_code':error_code}})
        source_code = None
    return source_code
    

Check link before expanding to save time

In [10]:
# Be sure to define current url
def get_links(soup, current_url):
    """Grabs all links from current_url's soup.

    Will pass on invalid links, links that match the avoid pattern, 
    and links that have already been grabbed.

    New Links will have open_seed: True meaning they can be used as a seed for a later step
    This value will be set to False when this page is used as a seed
    """
    # avoid pictures, files, and facebook pages,etc.
    avoid_pattern = re.compile('zillow\.com|coursera\.org|yelp\.com|flickr|tumblr|comments$|amazon.com|plus.google.com|linkedin.com|youtube.com|jobs.acm.org|vimeo.com|http://awards.acm.org|instagram.com|twitter.com|respond$|comment$|.pdf$|.png$|.jpg$|.jpeg$|.gif$|.xlsx$|wikipedia.org|facebook|\?share=|\?ref=footer_website|\?ref=footer_blog')
    for a_tag in soup.find_all('a'):
        href =a_tag.get('href')
        # Make sure that link is valid and not in avoid patterns, otherwise pass
        if href != None and http_pattern.match(href) and not avoid_pattern.search(href):
            # Check to see if the link has already been grabbed.
            if previously_grabbed(href):
                # if it has been previously grabbed AND it's an external link inc inbound_link_count and urls_pointed_here
                if external_link(href, current_url):
                    col.update_one({'url':href}, {'$inc': {'inbound_link_count':1}, '$push': {'urls_pointed_here':current_url}})
                pass
            # If not previously grabbed, expand link and check again
            else:
                source_code = make_request(href)
                # Try to connect via requests lib and if failed pass
                if source_code:
                    # failsafe for situations where url has been shortened, expands url
                    try:
                        url = source_code.url
                    except AttributeError:
                        requests_error_notice(0, href)
                        continue
                    # Query SQL or MongoDB to see if link has already been grabbed
                    if previously_grabbed(url):
                        # If the link points to a new web page increase that url's inbound_link_count by one
                        if external_link(url, current_url):
                            col.update_one({'url':url}, {'$inc': {'inbound_link_count':1}, '$push': {'urls_pointed_here':current_url}})
                        pass
                    # Only add a url to open_seed list if it has not been previously grabbed and it is valid
                    # If it is an external link, put inbound_link_count at 1    
                    else:
                        try:
                            blog_text = scrape_url(source_code)
                            update_blog_entry(url, blog_text, current_url)
                        except AttributeError:
                            requests_error_notice(0, href)
                            pass
                else:
                    pass
        else:
            pass

In [11]:
# Helper function for get_links
def external_link(url, current_url):
    """Checks to see if a link is internal or external.
    This is for initializing a mock page-rank system:
        Mock page-rank system will tally up the number
        of incomming links from external web pages
        across scraped pages."""
    parsed_url = urlparse(url)
    parsed_current_url = urlparse(current_url)
    # if both the scheme and the netloc match between sites it is an internal link, return false
    if parsed_url.scheme == parsed_current_url.scheme and parsed_url.netloc == parsed_current_url.netloc:
        return False
    else:
        return True

In [12]:
# Helper function for get_links
def previously_grabbed(url):
    """Check to see if url is already in DataBase
    If it has been previously scraped it will return True and webcrawler will act accordingly
    If it exists more than once, something has gone wrong and an e-mail will be sent out to notify users
    If it has not yet been scrapped it returns false
    """
    if col.find({'url':url}).count() == 1:
        return True
    
    # if the url exists multiple times e-mail out notification, something went wrong!
    elif col.find({'url':url}).count() > 1:
        duplicate_count = col.find({'url':url}).count()
        duplicate_notice(db_type, db, col, url, duplicate_count)
        return True
    
    else:
        return False

In [13]:
def scrape_url(source_code):
    """Takes a live url and it's BeautifulSoup object and pulls all the text from the body
    Does not pull from inputs, scripts, and noscripts (to avoid JavaScript)
    Returns the text as blog_text
    """
    soup = BeautifulSoup(source_code.text, "lxml", parse_only=only_body_tag)
    blog_text = []
    # still grabbing some javascript
    for child in soup.body.children:
        if child.name != None and child.name != 'script' and child.name != 'input' and child.name != 'noscript' and child.name != 'style' and child.name!= 'option':
            blog_text.append(child.get_text(' ', strip=True))
    space = ' '
    blog_text = space.join(blog_text)
    blog_text.encode('utf-8')
    
    return blog_text

In [14]:
# Needs to be Reworked! But for now we're turning it off to accept all pages
# Runs inside of update_blog_entry
def quality_check(url, blog_text):
    """Takes a url the scraped_text and passes it through a quality check.
    
    For this project that quality check is a Naive Bayes Classifier trained on categorized
    blog posts from initial scraping.
    
    If the blog_text meets relevancy criteria, return True, else return False
    """
    # Insert Naive Bayes classification here, if pass return True if fail return False
    """Example psuedo-code:
    category = nb_classifier.predict(blog_text)
    if category == 1:
        return True
    else:
        return False"""
    return True

In [15]:
def update_blog_entry(url, blog_text, current_url):
    """If blog_text passes the quality check, update it's entry in mongo_db
    Otherwise set open_seed to False and drop blog_text information
    
    For relevant web pages:
    
    hand_reviewed:
    --- True if this entry has been hand reviewed
    --- False if this entry has not been hand reviewed
    
    quality:
    --- True if this web page passed quality check
    --- False if this web page failed quality check
    """
    if quality_check(url, blog_text):
        if external_link(url, current_url):
            col.insert_one(
                {'url': url,
                 'text': blog_text,
                 'hand_reviewed': False,
                 'quality': True,
                 'open_seed':True,
                 'inbound_link_count':1,
                 'urls_pointed_here':[current_url]
                }
            )
        else:
            col.insert_one(
                {'url': url,
                 'text': blog_text,
                 'hand_reviewed': False,
                 'quality': True,
                 'open_seed':True,
                 'inbound_link_count':0
                }
            )
    else:
        col.insert_one(
            {'url':url,
            'hand_reviewed':False,
            'quality': False
            }
        )

In [117]:
max_iterations = 225
def run_crawler(max_iterations, location):
    for _ in range(max_iterations):
        current_url = get_seed()
        source_code = make_request(current_url)
        if source_code:
            soup = BeautifulSoup(source_code.text, "lxml", parse_only=only_body_tag)
            get_links(soup, current_url)
        else:
            pass
    termination_notice(location)

In [142]:
run_crawler(1, location)

In [16]:
db.backup.find()[0]

{'_id': ObjectId('593f0200f7e49c1cd22aea29'),
 'hand_reviewed': False,
 'inbound_link_count': 8,
 'open_seed': True,
 'quality': True,
 'text': " Business School Courses & Directory Business Schools Directory & Campus and Online mba Degrees Browse our MBA Info Library Home MBA Subjects MBA Schools Specialty MBAs MBA Library About Blog Explore MBA Subjects Accounting Business Administration Business Economics Communication Consulting Criminal Justice e-Business/e-Commerce Economics Education Engineering Entrepreneurship Executive / Mgt. Finance General Management Global Management Health Care / Administration Hospitality and Tourism Human Resources Industrial Management Information Systems / IT International Business Knowledge Management Leadership Marketing Media Nonprofit and Goverment Mgt. Operations Management Organizational Mgt. Project Management Real Estate Restaurant Enterprise Management Enterprise Management Service Management Sports Management Technology Management Telecommun

In [81]:
# Update all entries in the collection
# col.update_many({}, {'$set': {'inbound_link_count':0}})
# col.update_many({},{'$set':{'open_seed': True}})

# Should be 225 if no changes have been made yet:
# col.find({'open_seed':True}).count()

In [35]:
# remove a field-value pair, useful for keeping storage free/size low
# col.update_one({'url':url},{'$unset': {'urls_pointed_here':""}})

In [ ]:
# Example Class from previous project for turning this into a class object
# class weather_date:
#     """Class for scraping weather_data from www.wunderground.com"""
#     def __init__(self, soup_obj, zipcode, date):
#         self.date = date
#         self.soup_obj = soup_obj
#         self.zipcode = zipcode
#         self.sunrise = self.get_sunrise(soup_obj)
#         self.sunset = self.get_sunset(soup_obj)
#         self.dew_point = self.get_dew_point(soup_obj)
#         self.events = self.get_events(soup_obj)
#         self.sea_level_pressure = self.get_sea_level_pressure(soup_obj)
#         self.temp = self.get_temp(soup_obj)
#         self.humidity = self.get_humidity(soup_obj)
#         self.degree_days = self.get_degree_days(soup_obj)
#         self.wind = self.get_wind(soup_obj)
#         self.precipitation = self.get_precipitation(soup_obj)
#         self.snow = self.get_snow(soup_obj)
#         self.moon_phase = self.get_moon_phase(soup_obj)
#         self.weather_dict = self.create_dict() 
    
#     # Preliminary cleaning of data
#     def clean(self, result):
#         pattern = re.compile(r'\\n|\\xa0|\\xc2|\\t|b\'|\\xb0|<\w+>|</\w+>|')
#         return re.sub(pattern,'', str(result.encode('utf8')))
    
#     def get_sunrise(self, soup):
#         sunrise = soup.find(text='Actual Time').find_next('td').text
#         return self.clean(sunrise)
    
#     def get_sunset(self, soup):
#         sunset = soup.find(text='Actual Time').find_next('td').find_next('td').text
#         return self.clean(sunset)
    
#     def get_dew_point(self, soup):
#         dew_point = soup.find(text='Dew Point').find_next('td').text
#         return self.clean(dew_point)
    
#     def get_events(self, soup):
#         events = soup.find(text='Events').find_next('td').text
#         return self.clean(events)
    
#     def get_sea_level_pressure(self, soup):
#         sea_level_pressure = soup.find(text='Sea Level Pressure').find_next(text='Sea Level Pressure').find_next('td').text
#         return self.clean(sea_level_pressure)
    
#     def get_temp(self, soup):
#         temp_regex = re.compile(' temperature', re.I)
#         temp_entries = soup.find_all(text = temp_regex)
#         temp_list = []
#         for record in temp_entries:
#             temp_list.append(self.clean(record.find_next('td').text))
#         return temp_entries, temp_list
    
#     def get_humidity(self, soup):
#         hum_regex = re.compile(' humidity', re.I)
#         hum_entries = soup.find_all(text = hum_regex)
#         temp_list = []
#         for record in hum_entries:
#             temp_list.append(record.find_next('td').text.encode('utf8'))
#         return hum_entries, temp_list
    
#     def get_degree_days(self, soup):
#         temp_list = []
#         degree_days_regex = re.compile('ing degree days', re.I)
#         degree_days = soup.find_all(text = degree_days_regex)
#         for record in degree_days:
#             temp_list.append(self.clean(record.find_next('td').text))
#         return degree_days, temp_list
    
#     def get_wind(self, soup):
#         wind_regex = re.compile(' speed', re.I)
#         wind_entries = soup.find_all(text = wind_regex)
#         wind_entries = wind_entries[:3]
#         temp_list = []
#         for record in wind_entries[:3]:
#             temp_list.append(self.clean(record.find_next('td').text))
#         return wind_entries, temp_list
    
#     def get_precipitation(self, soup):
#         precipitation_regex = re.compile('precipitation', re.I)
#         precipitation = soup.find_all(text=precipitation_regex)
#         precipitation = precipitation[1:-1]
#         temp_list = []
#         for record in precipitation:
#             temp_list.append(self.clean(record.find_next('td').text))
#         return precipitation, temp_list
    
#     def get_snow(self, soup):
#         snow_regex = re.compile('snow', re.I)
#         snow = soup.find(class_='history-table-grey-header', text=snow_regex)
#         snow_entries = ['Snow', 'Month to date snowfall', 'Since 1 July snowfall', 'Snow Depth']
#         snow_data = []
#         for x in range(5):
#             try:
#                 snow = snow.find_next(text=snow_regex)
#                 snow_entries.append(self.clean(snow.encode))
#                 snow_data.append(self.clean(snow.find_next('td').text))
#             except AttributeError:
#                 while len(snow_data) < len(snow_entries):
#                     snow_data.append(np.nan)
#             logging.debug('Error pulling snow for date %s' % date)
#         return snow_entries, snow_data
    
#     def get_moon_phase(self, soup):
#         moon_phase = soup.find(class_='phaseIcon').find_next('td').text
#         return self.clean(moon_phase)

#     def create_dict(self):
#         temp_dict = {}
#         temp_dict['Date'] = self.date
#         temp_dict['Zipcode'] = self.zipcode
#         temp_dict['Sunrise'] = self.sunrise
#         temp_dict['Sunset'] = self.sunset
#         temp_dict['Dew Point'] = self.dew_point
#         temp_dict['Events'] = self.events
#         temp_dict['Sea Level Pressure'] = self.sea_level_pressure
#         for n in range(len(self.temp[0])):
#             temp_dict[self.clean(self.temp[0][n])] = self.temp[1][n]
#         for n in range(len(self.humidity[0])):
#             temp_dict[self.clean(self.humidity[0][n])] = self.humidity[1][n]
#         for n in range(len(self.degree_days[0])):
#             temp_dict[self.clean(self.degree_days[0][n])] = self.degree_days[1][n]
#         for n in range(len(self.wind[0])):
#             temp_dict[self.clean(self.wind[0][n])] = self.wind[1][n]
#         for n in range(len(self.precipitation[0])):
#             temp_dict[self.clean(self.precipitation[0][n])] = self.precipitation[1][n]
#         for n in range(len(self.snow[0])):
#             temp_dict[self.clean(self.snow[0][n])] = self.snow[1][n]
#         temp_dict['Moon Phase'] = self.moon_phase
#         return temp_dict